<a href="https://colab.research.google.com/github/prnishtala/NLP/blob/main/FinalProject/END_Phase_1_CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries


In [1]:
import re
import spacy
import torch
import torchtext
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.data import Field, BucketIterator, TabularDataset, Dataset, Example
import random
import math
import time
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from torchtext.legacy import data
from torchtext.legacy.data import Field, BucketIterator
import torch.nn.functional as F
import keyword, token, tokenize
import spacy
import torchtext
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)
torch.set_deterministic(True)
np.random.seed(SEED)

/usr/local/lib/python3.7/dist-packages/torch/__init__.py:422: UserWarning: torch.set_deterministic is deprecated and will be removed in a future release. Please use torch.use_deterministic_algorithms instead
  "torch.set_deterministic is deprecated and will be removed in a future "


# Data Cleansing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

! cp "/content/drive/My Drive/NLP/english_python_data_raw.txt" english_python_data.txt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Read the file and remove the unwanted data
# Removed few anomalies manually to prevent over-engineering

with open('english_python_data.txt', 'r') as f:
  raw_data = f.readlines()

cleansed_data = []

for sentence in raw_data:
  # Remove the comments indicating beginning of Driver Codes
  if re.search(r'^\s*#\s+driver', sentence.lower()):
    continue
  # Remove the unwanted empty lines
  elif re.search(r'^\s*\n+\s*$', sentence.lower()):
    continue
  # Remove the comments in the pattern of "In[0-9]"
  elif re.search(r'^\s*#\s+in\[[0-9]+\]', sentence.lower()):
    continue
  # Remove the comments containing just the numbers
  elif re.search(r'^\s*#+\s*[0-9]*\s*\n', sentence.lower()):
    continue
  else:
    cleansed_data.append(sentence)


In [4]:
# Generate DataFrame from the Input Data

df = pd.DataFrame(columns = ['src', 'trg'])

src = ''
trg = ''
minimum_length = 30

for i,record in enumerate(cleansed_data):
  if record.startswith('#') and len(record) > minimum_length:
    df.loc[len(df)] = [src,trg]
    trg = ''
    src = record
  else:
    trg = trg + record

df = df.iloc[1:]
df = df.reset_index(drop=True)
 

In [6]:
df

,src,trg
0,,[]
1,write a python program to add two numbers,"[num1 = 1.5, num2 = 6.3, sum = num1 + num2, pr..."
2,write a python function to add two user provi...,"[def add_two_numbers(num1, num2):, sum = n..."
3,write a program to find and print the largest...,"[num1 = 10, num2 = 12, num3 = 14, if (num1 >= ..."
4,write a program to find and print the smalles...,"[num1 = 10, num2 = 12, num3 = 14, if (num1 <= ..."
...,...,...
4990,write a program to print bit wise or of two n...,"[a = 60, b = 13, c = a | b, print(""or"", c), , ]"
4991,write a program to print bit wise xor of two ...,"[a = 60, b = 13, c = a ^ b, print(""xor"", c), , ]"
4992,write a program to calculate binary ones comp...,"[a = 60, c = ~a, print(""binary ones complement..."
4993,write a program to binary left shift a number,"[c = a << 2, print(""binary left shift"", c), , ]"


# Tokenizing and Embedding

In [7]:
spacy_en = spacy.load('en')

In [8]:
SRC = Field(tokenize = 'spacy', 
            eos_token = 'end',
            init_token = '<sos>', 
            lower = True,
            batch_first = True)

TRG = Field(tokenize = 'spacy', 
            eos_token = 'end',
            init_token = '<sos>', 
            lower = True,
            batch_first = True)

fields = [("src", SRC), ("trg", TRG)]

In [9]:
from torchtext.legacy import data
from torchtext.legacy.data import Field, BucketIterator
fields = [("src", SRC), ("trg", TRG)]

examples = [data.Example.fromlist([df.src[i],df.trg[i]], fields) for i in range(df.shape[0])] 

# Preparing the Traning, Validation and Testing datasets
dataset = data.Dataset(examples, fields)

(train_data, valid_data, test_data) = dataset.split(split_ratio=[0.80, 0.10, 0.10], random_state=random.seed(SEED))

## Build Vocab

In [10]:
SRC.build_vocab(train_data, min_freq = 1)
TRG.build_vocab(train_data, min_freq = 1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
BATCH_SIZE = 128

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data), 
     sort = False,
     batch_size = BATCH_SIZE,
     device = device)

In [12]:
spacy_en = spacy.load('en')
def Tokenize(sentence):
  sentence = str(sentence).replace('\n', '\t\t')
  return [tok.text for tok in spacy_en.tokenizer(sentence)]

## Gensim Embeddings

In [13]:
# Using gensim for embedding Python syntaxes for Python code, gensim with Word2Vec model has be utilized and created a embedding with the same dimension ie 256.

import gensim
dim = 256
min_count = 2
window = 3
target = []
word2vec_vectors = []

for rec in df['trg'].values:
  rec_token = Tokenize(rec)
  target.append(rec_token)
word2vec_model = gensim.models.Word2Vec(target, size = dim, window = window, min_count = min_count)


for token, i in TRG.vocab.stoi.items():
  if token in word2vec_model.wv.vocab.keys():
    word2vec_vectors.append(torch.FloatTensor(word2vec_model[token]))
  else:
    word2vec_vectors.append(torch.zeros(dim))
TRG.vocab.set_vectors(TRG.vocab.stoi, word2vec_vectors, dim)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)


In [14]:
# Display the embeddings generated
TRG.vocab.vectors

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [15]:
word2vec_model.save('embeddings.txt')

# Model

In [16]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 250):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [17]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [18]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [19]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [20]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 trainedEmbedding,
                 max_length = 250):
        super().__init__()
        
        self.device = device
        
        self.trainedEmbedding = trainedEmbedding
        self.tok_embedding = nn.Embedding.from_pretrained(self.trainedEmbedding)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [21]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [22]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

# Training Utils

In [23]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
trainedEmbedding = torch.FloatTensor(TRG.vocab.vectors)

enc = Encoder(INPUT_DIM,
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device, 
              trainedEmbedding)

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [24]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim()>1 and not isinstance(m,nn.Embedding):
        nn.init.xavier_normal_(m.weight.data)

model.apply(initialize_weights);

In [25]:
def count_parameters(model):
    total_params = 0
    for p in model.parameters():
        if p.requires_grad:
            total_params+=p.numel()
    print(f"Model has {total_params:,} trainable parameters")

count_parameters(model)

Model has 7,705,107 trainable parameters


In [26]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
        if isinstance(criterion, nn.CTCLoss):
            output_lengths = output.shape[1]
            target_lengths = trg.shape[1]-1
            output = output.contiguous().permute(1,0,2)
            trg = trg[:,1:].contiguous()
            loss = criterion(output, trg, output_lengths, target_lengths)
        else:
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            loss = criterion(output, trg)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]

        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [27]:
def evaluate(model, iterator, criterion, metrics=None):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            if isinstance(criterion, nn.CTCLoss):
                output_lengths = torch.full(size=(output.shape[0],), fill_value = output.shape[1], dtype=torch.long)
                target_lengths = trg.shape[1]-1
                output = output.contiguous().permute(1,0,2)
                trg = trg[:,1:].contiguous()
                loss = criterion(output, trg, output_lengths, target_lengths)

            else:
                output = output.contiguous().view(-1, output_dim)
                trg = trg[:,1:].contiguous().view(-1)
                loss = criterion(output, trg)
                
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [28]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [29]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [30]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [31]:
import time
N_EPOCHS = 75
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 5s
	Train Loss: 8.215 | Train PPL: 3694.560
	 Val. Loss: 7.634 |  Val. PPL: 2066.789
Epoch: 02 | Time: 0m 5s
	Train Loss: 7.278 | Train PPL: 1448.787
	 Val. Loss: 7.610 |  Val. PPL: 2017.391
Epoch: 03 | Time: 0m 5s
	Train Loss: 7.122 | Train PPL: 1239.321
	 Val. Loss: 7.758 |  Val. PPL: 2340.751
Epoch: 04 | Time: 0m 5s
	Train Loss: 6.961 | Train PPL: 1054.944
	 Val. Loss: 7.750 |  Val. PPL: 2322.053
Epoch: 05 | Time: 0m 5s
	Train Loss: 6.801 | Train PPL: 898.694
	 Val. Loss: 7.696 |  Val. PPL: 2199.124
Epoch: 06 | Time: 0m 5s
	Train Loss: 6.709 | Train PPL: 819.516
	 Val. Loss: 7.679 |  Val. PPL: 2162.342
Epoch: 07 | Time: 0m 5s
	Train Loss: 6.588 | Train PPL: 726.419
	 Val. Loss: 7.669 |  Val. PPL: 2140.154
Epoch: 08 | Time: 0m 5s
	Train Loss: 6.496 | Train PPL: 662.247
	 Val. Loss: 7.572 |  Val. PPL: 1943.175
Epoch: 09 | Time: 0m 5s
	Train Loss: 6.372 | Train PPL: 585.301
	 Val. Loss: 7.548 |  Val. PPL: 1896.993
Epoch: 10 | Time: 0m 5s
	Train Loss: 6.273 | Train 

# Testing the Model

In [36]:
import spacy
def text2Python(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

# Sample Test

In [37]:
sourceText = 'write a python program to prints common letters in two input strings\n '

print(sourceText )

translation, attention = text2Python(sourceText , SRC, TRG, model, device)

for i in range(len(translation)):
  print(translation[i])

write a python program to prints common letters in two input strings
 
s1='python'
s2='schoolofai'
a=list(set(s1)&set(s2))
print("the common letters are:")
for i in a:
    print(i)

end


# Testing the code on 25 randomly generated samples

In [44]:
import random

for i in range(25):
  inf = random.randint(1,2500)
  src = vars(train_data.examples[inf])['src']

  print("\n\nQuestion: ", ' '.join(src));
  print("\n")
  
  translation, attention = text2Python(src , SRC, TRG, model, device)

  for i in range(len(translation)):
    print(translation[i])



Question:    100 write a python program that checks if a string is a pallindrome



def is_palindrome(st):
    st = st.lower()
    rev_st = st[::-1]
    try:
        assert rev_st == st
        return true
    except assertionerror:
        return false

st = "nitin"
print(is_palindrome(st))


end


Question:    write a python function to generate random number between 2 integers


def random_number(a, b):
    import random
    return random.randint(a, b)


end


Question:    write a python class to print all possible subsets from a set of distinct integers


class sub:
    def f1(self, s1):
        return self.f2([], sorted(s1))
    def f2(self, curr, s1):
        if s1:
            return self.f2(curr, s1[1:]) + self.f2(curr + [s1[0]], s1[1:])
        return [curr]
a=[2, 3, 5, 6, 4, 5]
print("subsets: ")
print(sub().f1(a))

end


Question:    write a python function to get prominent words from user test corpus using tfidf vectorizer


def get_words(corpus, new_doc, top=2):
    impo